# 일별 데이터 취합

In [1]:
'''
import dask.dataframe as dd
import os

# 2024년 9월 1일부터 9월 30일까지 데이터를 저장할 리스트
data_list = []

# 2024년 9월 1일부터 9월 30일까지 반복
for day in range(1, 31):
    # 파일 경로 생성
    file_name = f'./seoul_purpose/seoul_purpose_admdong3_final_202409{day:02d}.csv'
    
    # 파일 존재 여부 확인
    if os.path.exists(file_name):
        # Dask 데이터프레임 읽기
        df = dd.read_csv(file_name)

        # 전처리: 출근 목적, 외국인 시민명('한국') 필터링
        df_filtered = df[(df['move_purpose'] == 1) & (df['forn_citiz_nm'] == '한국')]

        # 필터링된 데이터 저장
        data_list.append(df_filtered)
    else:
        print(f'파일 없음: {file_name}')

# 리스트에 저장된 데이터프레임들을 하나로 병합
if data_list:
    final_df = dd.concat(data_list)

    # 병합된 결과를 저장하거나 처리
    final_df = final_df.compute()  # 필요한 경우 메모리로 가져오기
    print("모든 데이터를 성공적으로 병합했습니다.")
else:
    print("병합할 데이터가 없습니다.")

df = final_df.reset_index(drop=True)

df.to_csv('seoul_purpose_2409.csv',encoding='utf-8',index=False)

'''
print('완료')

완료


In [2]:
import pandas as pd
import dask.dataframe as dd

df =dd.read_csv('seoul_purpose_2409.csv')
df

,o_admdong_cd,d_admdong_cd,st_time_cd,fns_time_cd,in_forn_div_nm,forn_citiz_nm,move_purpose,move_dist,move_time,cnt,etl_ymd
npartitions=27,,,,,,,,,,,
,int64,int64,int64,int64,string,string,int64,float64,float64,float64,int64
,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...


In [3]:
import dask.dataframe as dd
import pandas as pd

# 행정동코드 로드 및 전처리
df_dong = pd.read_excel('./행정동코드/KIKcd_H.20240118(말소코드포함).xlsx')
df_dong = df_dong[['행정동코드', '읍면동명']].dropna().reset_index(drop=True)
df_dong.rename(columns={'행정동코드': 'hdong_cd', '읍면동명': 'hdong_nm'}, inplace=True)
# 뒤의 두 자리 "00" 제거 및 int64로 변환
df_dong["hdong_cd"] = df_dong["hdong_cd"].astype(str).str[:-2].astype('int64')

# Dask 데이터프레임으로 로드
df = df.rename(columns={
    'o_admdong_cd': 'origin_hdong_cd',
    'd_admdong_cd': 'dest_hdong_cd'
})

# pandas를 사용한 join을 위해 Dask에서 메모리 효율성을 고려하여 관리
df_dong_dd = dd.from_pandas(df_dong, npartitions=1)

# 출발지 행정동명 추가
df = df.merge(
    df_dong_dd.rename(columns={'hdong_cd': 'origin_hdong_cd', 'hdong_nm': 'origin_hdong_nm'}),
    on='origin_hdong_cd',
    how='left'
)

# 도착지 행정동명 추가
df = df.merge(
    df_dong_dd.rename(columns={'hdong_cd': 'dest_hdong_cd', 'hdong_nm': 'dest_hdong_nm'}),
    on='dest_hdong_cd',
    how='left'
)

# 결과 출력
result = df.compute()  # Dask 데이터프레임을 Pandas로 변환
result.head()    # 결과의 상위 5개 행 출력


,origin_hdong_cd,dest_hdong_cd,st_time_cd,fns_time_cd,in_forn_div_nm,forn_citiz_nm,move_purpose,move_dist,move_time,cnt,etl_ymd,origin_hdong_nm,dest_hdong_nm
0,11590680,41610560,720,920,내국인,한국,1,30046.0,102.0,2.57,20240901,신대방제2동,광남1동
1,11410640,11110530,840,900,내국인,한국,1,3373.0,18.0,2.20,20240901,홍제제3동,사직동
2,11710580,11710710,12,13,내국인,한국,1,1250.0,9.0,3.03,20240901,송파1동,잠실6동
3,11440610,11440555,1720,1720,내국인,한국,1,447.0,5.0,2.83,20240901,염리동,아현동
4,41210632,11470520,13,16,내국인,한국,1,9106.0,144.0,3.87,20240901,하안2동,목2동


In [4]:
# 서울 내에서 출근하는 인원으로 필터링
df = result[
    result["origin_hdong_cd"].astype(str).str.startswith("11") &
    result["dest_hdong_cd"].astype(str).str.startswith("11")
].reset_index(drop=True)

df

,origin_hdong_cd,dest_hdong_cd,st_time_cd,fns_time_cd,in_forn_div_nm,forn_citiz_nm,move_purpose,move_dist,move_time,cnt,etl_ymd,origin_hdong_nm,dest_hdong_nm
0,11410640,11110530,840,900,내국인,한국,1,3373.0,18.0,2.20,20240901,홍제제3동,사직동
1,11710580,11710710,12,13,내국인,한국,1,1250.0,9.0,3.03,20240901,송파1동,잠실6동
2,11440610,11440555,1720,1720,내국인,한국,1,447.0,5.0,2.83,20240901,염리동,아현동
3,11740550,11740550,0,0,내국인,한국,1,100.0,8.0,3.75,20240901,고덕제1동,고덕제1동
4,11380625,11410585,11,12,내국인,한국,1,5575.0,42.0,3.04,20240901,역촌동,신촌동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8623375,11110615,11110615,840,920,내국인,한국,1,765.0,39.0,8.97,20240930,종로1.2.3.4가동,종로1.2.3.4가동
8623376,11110650,11110615,800,820,내국인,한국,1,2130.0,26.0,10.53,20240930,혜화동,종로1.2.3.4가동
8623377,11380520,11500615,6,800,내국인,한국,1,10822.0,62.0,3.62,20240930,불광제1동,우장산동
8623378,11590540,11140520,720,740,내국인,한국,1,7691.0,33.0,2.93,20240930,상도제2동,소공동


In [5]:
df = df[['origin_hdong_cd','dest_hdong_cd','origin_hdong_nm','dest_hdong_nm','etl_ymd','st_time_cd','fns_time_cd','move_dist','move_time','cnt']]
df

,origin_hdong_cd,dest_hdong_cd,origin_hdong_nm,dest_hdong_nm,etl_ymd,st_time_cd,fns_time_cd,move_dist,move_time,cnt
0,11410640,11110530,홍제제3동,사직동,20240901,840,900,3373.0,18.0,2.20
1,11710580,11710710,송파1동,잠실6동,20240901,12,13,1250.0,9.0,3.03
2,11440610,11440555,염리동,아현동,20240901,1720,1720,447.0,5.0,2.83
3,11740550,11740550,고덕제1동,고덕제1동,20240901,0,0,100.0,8.0,3.75
4,11380625,11410585,역촌동,신촌동,20240901,11,12,5575.0,42.0,3.04
...,...,...,...,...,...,...,...,...,...,...
8623375,11110615,11110615,종로1.2.3.4가동,종로1.2.3.4가동,20240930,840,920,765.0,39.0,8.97
8623376,11110650,11110615,혜화동,종로1.2.3.4가동,20240930,800,820,2130.0,26.0,10.53
8623377,11380520,11500615,불광제1동,우장산동,20240930,6,800,10822.0,62.0,3.62
8623378,11590540,11140520,상도제2동,소공동,20240930,720,740,7691.0,33.0,2.93


In [6]:
import pandas as pd

# 시간 변환 함수
def convert_time(time):
    # 4자리 숫자 그대로 유지
    if time >= 1000 or (time >= 700 and time <= 940) or (time >= 1700 and time <= 1940):
        return f"{time:04d}"
    # 나머지 값은 100 곱해서 4자리로 변환
    else:
        return f"{time * 100:04d}"

# 변환 적용
df['st_time_cd'] = df['st_time_cd'].apply(convert_time)
df['fns_time_cd'] = df['fns_time_cd'].apply(convert_time)

df

C:\Users\samsung-user\AppData\Local\Temp\ipykernel_8860\206906479.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['st_time_cd'] = df['st_time_cd'].apply(convert_time)
C:\Users\samsung-user\AppData\Local\Temp\ipykernel_8860\206906479.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fns_time_cd'] = df['fns_time_cd'].apply(convert_time)


,origin_hdong_cd,dest_hdong_cd,origin_hdong_nm,dest_hdong_nm,etl_ymd,st_time_cd,fns_time_cd,move_dist,move_time,cnt
0,11410640,11110530,홍제제3동,사직동,20240901,0840,0900,3373.0,18.0,2.20
1,11710580,11710710,송파1동,잠실6동,20240901,1200,1300,1250.0,9.0,3.03
2,11440610,11440555,염리동,아현동,20240901,1720,1720,447.0,5.0,2.83
3,11740550,11740550,고덕제1동,고덕제1동,20240901,0000,0000,100.0,8.0,3.75
4,11380625,11410585,역촌동,신촌동,20240901,1100,1200,5575.0,42.0,3.04
...,...,...,...,...,...,...,...,...,...,...
8623375,11110615,11110615,종로1.2.3.4가동,종로1.2.3.4가동,20240930,0840,0920,765.0,39.0,8.97
8623376,11110650,11110615,혜화동,종로1.2.3.4가동,20240930,0800,0820,2130.0,26.0,10.53
8623377,11380520,11500615,불광제1동,우장산동,20240930,0600,0800,10822.0,62.0,3.62
8623378,11590540,11140520,상도제2동,소공동,20240930,0720,0740,7691.0,33.0,2.93


In [7]:
df.to_csv('2409_생활이동.csv', encoding='utf-8', index=False)